# Polars - Blazingly fast DataFrames in 2023

It has been over 2 years already since [I created my first notebook about Polars](https://github.com/r-brink/polars-tutorial/blob/master/polars-tutorial.ipynb) ([Medium post here](https://r-brink.medium.com/introduction-to-polars-ee9e638dc163)). Back then a small project started by Ritchie Vink with promising performance. 2 years later, the project has over 10.000 stars on [Github](https://github.com/pola-rs/polars) and is still leading many performance benchmarks. 

In this updated tutorial we will revisit the original analyses and rewrite the queries following the latest version of Polars and the Polars book.

The structure of this repo:


> Remarks written in the earlier notebook are marked like this (as a quote)


```python
import polars as pl

# code snippets from the previous notebook are commented out in the same cell
# so you can compare them easily
```

## Installing Polars

The first notebook ran with Polars `0.7.0`. Definitely time for an update.

First install Polars in your virtual environment. In this notebook we will work with Polars version 0.15.7. Add this to the pip install to ensure all cells run.

```shell
pip install polars==0.15.7
```

### Importing relevant packages and loading the data

In [ ]:
import polars as pl

import matplotlib.pyplot as plt
%matplotlib inline

Nothing changed here. Still `import polars as pl`. 

> Polars already offers many functionalities that we are already familiar if you have worked with Pandas before. We can find an overview, including examples (for most), in the reference guide.

Find the dataset that we are going to use here: [Winemag-dataset](https://medium.com/p/48db12c0f067/edit#%20find%20the%20dataset%20here:%20https://www.kaggle.com/zynicide/wine-reviews/?select=winemag-data_first150k.csv)

In [ ]:
df = pl.read_csv('winemag-data_first150k.csv')

### Basic dataframe inspection

In [ ]:
df.shape

In [ ]:
df.columns

Nothing has changed so far, until now. Instead of the Pandas-like syntax we used two years ago. We will use the expressions, as recommended by Polars.

> We strongly recommend selecting data with expressions for almost all use cases. Square bracket indexing is perhaps useful when doing exploratory data analysis in a terminal or notebook when you just want a quick look at a subset of data. ~ [Polars User Guide](https://pola-rs.github.io/polars-book/user-guide/)

In [ ]:
display(df.select([
    pl.col('*').sample(n=5)
]))

#df.sample(n=5)

We could still use the old way, as it is shorter. However, we can easily extend this expression to include more information if we want. It also helps us to solidify to work with expression in our workflow.

> The dataset has a lot to offer. With 11 variables and over 150k rows, there is a lot of data to analyse. We see a couple of variables that are interesting to look into, like price, country, points.

To select a column, we use the following expression.

In [ ]:
df.select(
    pl.col("price")
).limit(10)

# Earlier we used the following:
#   df['price']
# Select by index is considered an anti-pattern in Polars 

## Removing nulls

>Before we continue we want to have a closer look at if there are any nulls in the dataset.

In [ ]:
df.select(
    pl.col("*").null_count()
)

# Earlier we used the following: 
#   df.null_count()   

Similar to our original notebook we will fill the nulls with the mean for the following reason.

> It seems that around a little less than 10% of the price variable has no value. We can either drop the rows with missing values or fill them. In this article, we will choose to use the mean as filling strategy.

In [ ]:
#TODO: look at this cell, as it does not run
df = df.with_column(
        pl.col("price").fill_null(strategy="mean").alias('price')
)

print(df.select('price').null_count())

## Some analyses

> The next step is to dive in a little deeper and have a closer look at the dataset with some more complex functions.
> The goal that we want to achieve in the following part is to have a closer look at the countries and how they compare in terms of price and points.

In [ ]:
df.select([
        pl.median("price").alias("median price"),
        pl.min("price").alias("min price"),
        pl.max("price").alias("max price"),
        pl.mean("price").alias("mean price")
    ]
)

# Analyses of wine prices
#print(f'Median price: {df["price"].median()}')
# print(f'Average price: {df["price"].mean()}')
# print(f'Maximum price: {df["price"].max()}')
# print(f'Minimum price: {df["price"].min()}')

In [ ]:
df.select([
        pl.median("points").alias("median points"),
        pl.min("points").alias("min points"),
        pl.max("points").alias("max points"),
        pl.mean("points").alias("mean points")
    ]
)

# Analyses of wine points
# print(f'Median points: {data["points"].median()}')
# print(f'Average points: {data["points"].mean()}')
# print(f'Maximum points: {data["points"].max()}')
# print(f'Minimum points: {data["points"].min()}')

Or we can just use the `describe()` function for quick statistics about our dataframe.

In [ ]:
df.describe()

> The minimum number of points shows that there is no such thing as bad wine.

## Filtering

In [ ]:
df.select([
    pl.col("country").unique()
])

# data['country'].unique().to_list()

From our earlier notebook

> There are two strange values in our dataset: an undefined country ("") and a country called 'US-France'.

In [ ]:
df.filter(
    (pl.col('country') == 'US-France') |
    (pl.col('country').is_null())
)

> There are only 6 of them, so it is safe to drop them.

In [ ]:
df = df.filter(
    (pl.col('country').is_not_null()) &
    (pl.col('country') != 'US-France')
)

# data = data[(data['country'] != '') & (data['country'] != 'US-France')]

> Time to look into the countries that produces the best wine according to the points and has the highest price for a bottle.

In [ ]:
#TODO: look at this cell, as it is not running. This error shows up a lot in this notebook, and makes it not possible to run the rest of the notebook.
df.groupby('country').agg(
    pl.col('points').mean().alias('points_mean')
).sort(by='points_mean', reverse=True)

# We group by country, select the `points` variable 
# and call the mean to see the average number of points. 
# After that we sort the list by 'average points'. 

# data.groupby('country')
#       .select('points').mean()
#       .sort(by_column='points_mean', reverse=True)

> England is leading the list for the best wines. Wonder how they think about that on the other side of the Canal in France.

In [ ]:
df.groupby('country').agg(
    pl.col('price').max().alias('price_max')
).sort(by='price_max', reverse=True)

# data.groupby('country')
#   .select('price').max()
#   .sort(by_column='price_max', reverse=True)

### Plotting with Polars

> To get a better insight into the differences it always helps to have some nice plots. Where Pandas has a plotting functionality build in, we have to rely on our Matplotlib skills for Polars. We focus on the top 15 countries.

In [ ]:
top_15_countries = df.groupby('country').agg(
    pl.col('points').mean().alias('points_mean')
).sort(by='points_mean', reverse=True).limit(15)

top_15_countries

# Get a list of the top 15 countries by taking the first 15 rows 
# of the groupby that we did earlier
# 
# top_15_countries = data.groupby('country')
#       .select('points').mean()
#       .sort(by_column='points_mean', reverse=True)[:15][0]

In [ ]:
df_top15 = top_15_countries.join(df, on='country', how='left')

df_top15

# df_top15 = pl.DataFrame({'country': top_15_countries}).join(data, on='country', how='left')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

for i, country_df in enumerate(df_top15.partition_by(groups="country")):
    country_name = country_df.select("country")[0, 0]
    ax.boxplot(country_df.select('points'), labels=[country_name], positions=[i])

plt.xticks(rotation=90)
plt.xlabel('Countries')
plt.ylabel('Average points')
plt.show()


# fig, ax = plt.subplots(figsize=(15, 5))

# for i, x in enumerate(df_top15['country'].unique()):
#     ax.boxplot(df_top15[df_top15['country'] == x]['points'], labels=[str(x)], positions=[i])

## Time to go Lazy

> The lazy API offers a way to optimise your queries, similar to Spark. The major benefit over spark is that we don't have to set up our environment and can therefore continue working from our notebook.

> More information can be found in the [Polars-book](https://ritchie46.github.io/polars-book/lazy_polars/intro.html)

In [ ]:
lazy_df = pl.scan_csv('winemag-data_first150k.csv')

lazy_df

# lazy_df = pl.scan_csv('winemag-data_first150k', ignore_errors=True)

# if you install Graphviz you will see the Query Plan 

> Printing the type returns 'polars.lazy.LazyFrame' indicating the data is available to us. On to the Groupby `country` and find the average `points` to compare with the eager API that we used earlier.

> Similar to the filters that we did with the eager API we are going to filter the unknown and 'US-France' values in the `country` variable first.

In [ ]:
lazy_df.filter(
    (pl.col('country').is_not_null()) &
    (pl.col('country') != 'US-France')
)

# we can see that the query is almost the same
# however this query only returns a query plan

> As we can see nothing happens right away. From the documentation: '_This is due to the lazyness, nothing will happen until specifically requested. This allows Polars to see the whole context of a query and optimize just in time for execution._'

> To actually see the results we can do two things: `collect()` and `fetch()`. The difference is that `fetch` takes the first 500 rows and then runs the query, whereas `collect` runs the query over all the results. Below we can see the differences for our case.

In [ ]:
lazy_df.filter(
    (pl.col('country').is_not_null()) &
    (pl.col('country') != 'US-France')
).fetch()

In [ ]:
lazy_df.filter(
    (pl.col('country').is_not_null()) &
    (pl.col('country') != 'US-France')
).collect()

We can see from the shapes that `fetch` catches 500 rows and `collect` retrieves all the rows.

In [ ]:
lazy_df.groupby('country').agg(
    pl.col('points').mean().alias('points_mean')
).sort(by='points_mean', reverse=True).collect()

#  lazy_df = (
#     lazy_df
#     .groupby('country')
#     .agg([pl.mean('points').alias('avg_points')])
#     .sort("avg_points", reverse=True)
# )

### Out of Core [NEW]

What if you dataset doesn't fit in memory? This example is rather small, but in this day and age it is not unlikely that you are working on datsets that don't fit in memory any more. Polars offers a very easy way to work with that. 

Pretend that our dataset is not ~50MB, but 50GB. What can we do to, for example, Groupby country and do some calculations:

Not much changes, except that in `collect()` we add: `streaming=True` 

In [ ]:
lazy_df.groupby('country').agg(
    pl.col('points').mean().alias('points_mean')
).sort(by='points_mean', reverse=True).collect(streaming=True)

## Output

> We have got the output that we are looking for. Polars offers several ways to output our analyses, even to other formats useful for further analyses (e.g. pandas dataframe (`to_pandas()`) or numpy arrays (`to_numpy()`).

In [ ]:
lazy_df.collect().write_csv('results.csv')

#  lazy_df.collect().to_csv('results.csv')

## Final words from 2022

> Polars is a new package that is gaining a lot of attention. At the time of writing this article, it has gathered more than 1300 stars on Github, which is impressive looking at the fact that is around for less than a year. It offers almost all the functions that we need to manipulate our dataframe. Next to that, it offers a lazy API that helps us optimising our queries before we execute them. Although we didn't touch it is in this article, the benchmark of H20 shows that it is super efficient and fast. Especially with larger datasets it becomes worthwhile to look into the benefits that the lazy API has to offer.

This is what I wrote two years ago. 1300 stars. In 2022, while I am writing this, the project has collected 10.4k stars on Github and it is number 4 trending project on Github.

There are a lot of improvements happening under the hood and in the APIs. 

For more information, check the [Polars' Github page](https://github.com/pola-rs/polars). Here you can find links to benchmark's, the Polars book or the Polars Discord. 

I highly recommend joining the Discord. There is a lot of activity and many people are happy to help you with your specific questions.